# Pipeline ETL

## Extração


Em ETL/extração.py, os dados são extraídos dos arquivos .CSV da prefeitura que estão na pasta data/

Utilizando a bibilioteca Pandas, criamos um dataframe para cada arquivo entre 2006 e 2017, armazenamos todos em uma lista e depois os concatenamos e salvamos em um único grande arquivo .csv que será transformado posteriormente.


In [5]:
import pandas as pd
from pathlib import Path

pasta = Path(r"../data")

# Usa glob() do objeto Path de pathlib para conseguir o path para todos os arquivos CSV entre 2006 e 2017
arquivos_csv = sorted(
    [arq for arq in pasta.glob("recife-dados-despesas-*.csv") 
     if "2006" <= arq.stem.split('-')[-1] <= "2017"]
) 

lista_dfs = []

# Tabelas são carregads como dataframes e armazenadas na lista
for arq in arquivos_csv:
    data_frame = pd.read_csv(arq, sep=';', encoding='utf-8') 
    lista_dfs.append(data_frame)
    print(f"Processando arquivo: {arq}")

# Concatenação das tabelas na lista
df_final = pd.concat(lista_dfs) 

# Salva dataframe unificado para transformação
df_final.to_csv("../despesas_recife.csv",index=False)
print("\n Tabela unificada disponível em despesas_recife.csv para transformação.")


Processando arquivo: ..\data\recife-dados-despesas-2006.csv
Processando arquivo: ..\data\recife-dados-despesas-2007.csv
Processando arquivo: ..\data\recife-dados-despesas-2008.csv
Processando arquivo: ..\data\recife-dados-despesas-2009.csv
Processando arquivo: ..\data\recife-dados-despesas-2010.csv
Processando arquivo: ..\data\recife-dados-despesas-2011.csv
Processando arquivo: ..\data\recife-dados-despesas-2012.csv
Processando arquivo: ..\data\recife-dados-despesas-2013.csv
Processando arquivo: ..\data\recife-dados-despesas-2014.csv
Processando arquivo: ..\data\recife-dados-despesas-2015.csv
Processando arquivo: ..\data\recife-dados-despesas-2016.csv
Processando arquivo: ..\data\recife-dados-despesas-2017.csv

 Tabela unificada disponível em despesas_recife.csv para transformação.


## Transformação


Em ETL/transformação.py os dados são transformados, garantindo consistência dos dados antes destes serem carregados no banco de dados Postgres.

Primeiramente carregamos o CSV unificado criado no primeiro passo e indentificamos as colunas cujos elementos devem ser valores inteiros(Colunas relacionados a anos, meses e códigos de identificação) e aquelas colunas cujos valores devem ser números decimais (Valores monetários)

In [8]:
import pandas as pd

# Carrega dataframe unificado
df = pd.read_csv("../despesas_recife.csv", encoding='utf-8', low_memory=False)
print("Tabela despesas_recife.csv carregada.")

# Teste se existia alguma valor faltando em alguma coluna da tabela --- resultado 0 valores faltando nas colunas
# assim não foi preciso tratar valores faltantes
# print(df.isnull().sum())


COL_INT = ('empenho_ano', 'ano_movimentacao', 'mes_movimentacao', 'orgao_codigo',
            'grupo_despesa_codigo','modalidade_aplicacao_codigo','elemento_codigo',
            'subelemento_codigo','funcao_codigo','subfuncao_codigo','programa_codigo',
            'acao_codigo','fonte_recurso_codigo','empenho_numero','subempenho', 'credor_codigo',
            'modalidade_licitacao_codigo')

# Opta-se por numeric ao invés de float para evitar problemas com arredondamento e conseguir mais precisão
COL_NUMERIC = ('valor_empenhado', 'valor_liquidado', 'valor_pago')


Tabela despesas_recife.csv carregada.


Em seguida, efetuamos as transformações. Removemos espaços vazios e tornamos todos os caracteres de todas as colunas minúsculos para garantir que não haja uma repetição errônea de dados (Exemplo: Várias entradas para o mesmo Orgão só porque algumas dessas entradas têm uma quantidade diferente de espaços brancos.)

Então, convertemos as colunas previamente identificadas para int e numeric. No caso de numeric, como descobrimos uma inconsistência em como os valores monetários são armazenadas na tabela (Pré-2016: separação de decimais com . Pós-2016: separação de decimais com ,) primeiro trocamos ',' por '.' nas colunas de dinheiro pós-2016 e só então convertemos para numeric.

In [9]:
# Padronização das colunas
print("Padronizando células")
df.columns = (
    df.columns
    .str.strip()         # Remove espaços vazios no começo e fim
    .str.lower()         # Deixa tudo minúsculo
    .str.replace(" ", "_")  # Substitui espaços por underline
)

for coluna in COL_INT:
    print("Transformando coluna de inteiros:", coluna)
    df[coluna] = df[coluna].astype(int)

# Erro na transformação, a partir de 2016 'valor_empenhado', 'valor_liquidado', 'valor_pago' começaram a vir
# com , e não com . como nos anos anteriores
for coluna in COL_NUMERIC:
    print("Transformando coluna numérica:", coluna)
    # Para os anos a partir de 2016, troca vírgula por ponto antes de converter
    mask_2016 = df['ano_movimentacao'] >= 2016
    df.loc[mask_2016, coluna] = (
        df.loc[mask_2016, coluna]
        .astype(str)
        .str.replace('.', '', regex=False)   # Remove separador de milhar, se existir
        .str.replace(',', '.', regex=False)
    )

    # Aqui converte para numeric e errors='coerce' faz valores inválidos virarem NAN por precaução
    df[coluna] = pd.to_numeric(df[coluna],errors='coerce')


Padronizando células
Transformando coluna de inteiros: empenho_ano
Transformando coluna de inteiros: ano_movimentacao
Transformando coluna de inteiros: mes_movimentacao
Transformando coluna de inteiros: orgao_codigo
Transformando coluna de inteiros: grupo_despesa_codigo
Transformando coluna de inteiros: modalidade_aplicacao_codigo
Transformando coluna de inteiros: elemento_codigo
Transformando coluna de inteiros: subelemento_codigo
Transformando coluna de inteiros: funcao_codigo
Transformando coluna de inteiros: subfuncao_codigo
Transformando coluna de inteiros: programa_codigo
Transformando coluna de inteiros: acao_codigo
Transformando coluna de inteiros: fonte_recurso_codigo
Transformando coluna de inteiros: empenho_numero
Transformando coluna de inteiros: subempenho
Transformando coluna de inteiros: credor_codigo
Transformando coluna de inteiros: modalidade_licitacao_codigo
Transformando coluna numérica: valor_empenhado
Transformando coluna numérica: valor_liquidado
Transformando co

Por fim, asseguramos que todas as demais colunas sejam string e armazenamos o dateframe transformado em outro arquivo csv para o carregamento no banco de dados.

In [37]:
# Pega o conjunto de todas as colunas
todas_colunas = set(df.columns)

# Subtrai as colunas numéricas e inteiras
colunas_string = todas_colunas - set(COL_INT) - set(COL_NUMERIC)

# Converte as colunas restantes para string
for coluna in colunas_string:
    df[coluna] = (
        df[coluna]
        .astype(str)                # Garante tipo string
        .str.strip()                 # Remove espaços no início e no fim
        .str.lower()                 # Converte para minúsculas
        .str.replace(r"\s+", "_", regex=True)  # Substitui múltiplos espaços por um só
    )

# Salvando tratamento em um novo arquivo csv
df.to_csv("despesas_recife_tratadas.csv", index=False, encoding='utf-8')
print("\n Tabela transformada disponível em despesas_recife_tratadas.csv para carregamento.")


 Tabela transformada disponível em despesas_recife_tratadas.csv para carregamento.


## Carregamento

Inicialmente configuramos a conexão com o banco de dados postgres utilizando SQLAlchemy e váriaveis definidas no arquivo .env

In [ ]:
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os

load_dotenv()

DATABASE_URL = f"postgresql+psycopg2://{os.getenv('DATABASE_USER')}:{os.getenv('DATABASE_PASSWORD')}@{os.getenv('DATABASE_HOST')}:{os.getenv('DATABASE_PORT')}/{os.getenv('DATABASE_NAME')}"
engine = create_engine(DATABASE_URL)

with engine.begin() as conn:
    RESULT = conn.execute(text("SELECT NOW();")) 
    for row in RESULT:
        print(f"Connexão feita com sucesso: {row[0]}")

Connexão feita com sucesso: 2025-08-08 20:47:41.297317-03:00


Em ETL/carregamento.py realizamos o carregamento dos dados tratados para o postgres. Primeiramente definimos o tipo que cada coluna deve ter no banco de dados e então utilizamos o método .to_sql da biblioteca pandas em conjunta com a conexão ao banco de dados Postgres feita com SQLAlchemy para realizar o upload da tabela transformada.

In [47]:
from sqlalchemy.types import Integer, Numeric, String
import pandas as pd

# Carrega o CSV JÁ TRATADO
df = pd.read_csv("despesas_recife_tratadas.csv", encoding='utf-8')

# Apenas mapeia tipos do Pandas para o PostgreSQL evitando que ele interprete sozinho e converta os valores que eu já havia transformado
tipos_colunas_sqlalchemy = {
    # Colunas inteiras
    'ano_movimentacao': Integer(),
    'mes_movimentacao': Integer(),
    'orgao_codigo': Integer(),
    'grupo_despesa_codigo': Integer(),
    'modalidade_aplicacao_codigo': Integer(),
    'elemento_codigo': Integer(),
    'subelemento_codigo': Integer(),
    'funcao_codigo': Integer(),
    'subfuncao_codigo': Integer(),
    'programa_codigo': Integer(),
    'acao_codigo': Integer(),
    'fonte_recurso_codigo': Integer(),
    'empenho_ano': Integer(),
    'empenho_numero': Integer(),
    'subempenho': Integer(),
    'credor_codigo': Integer(),
    'modalidade_licitacao_codigo': Integer(),

    # Colunas numéricas
    'valor_empenhado': Numeric(18, 2),
    'valor_liquidado': Numeric(18, 2),
    'valor_pago': Numeric(18, 2),

    #  colunas que são strings
    'orgao_nome': String(255),
    'unidade_codigo': String(50),  # Códigos não numéricos são strings
    'unidade_nome': String(255),
    'categoria_economica_codigo': String(50),
    'categoria_economica_nome': String(255),
    'grupo_despesa_nome': String(255),
    'modalidade_aplicacao_nome': String(255),
    'elemento_nome': String(255),
    'subelemento_nome': String(255),
    'funcao_nome': String(255),
    'subfuncao_nome': String(255),
    'programa_nome': String(255),
    'acao_nome': String(255),
    'fonte_recurso_nome': String(255),
    'empenho_modalidade_nome': String(255),
    'empenho_modalidade_codigo': String(50),
    'indicador_subempenho': String(50),
    'credor_nome': String(255),
    'modalidade_licitacao_nome': String(255)
}

# Envia ao banco (os dados já estão tratados)
print("Carregamento de dados iniciado")
df.to_sql(
    name="despesas_recife",
    con=engine,
    if_exists="replace",
    index=False,
    dtype=tipos_colunas_sqlalchemy
)

print("Carregamento de dados finalizado")

Carregamento de dados iniciado
Carregamento de dados finalizado


In [49]:
query_df = pd.read_sql("SELECT * FROM despesas_recife TABLESAMPLE SYSTEM (1) WHERE ano_movimentacao BETWEEN 2006 AND 2017 ORDER BY VALOR_PAGO DESC LIMIT 10;", engine)
query_df

,ano_movimentacao,mes_movimentacao,orgao_codigo,orgao_nome,unidade_codigo,unidade_nome,categoria_economica_codigo,categoria_economica_nome,grupo_despesa_codigo,grupo_despesa_nome,...,empenho_numero,subempenho,indicador_subempenho,credor_codigo,credor_nome,modalidade_licitacao_codigo,modalidade_licitacao_nome,valor_empenhado,valor_liquidado,valor_pago
0,2011,9,14,"secretaria_de_educação,_esporte_e_lazer",14.01,"secretaria_de_educação,_esporte_e_lazer_-_admi...",3,despesas_correntes,1,pessoal_e_encargos_sociais,...,319,89,x,0,credor_não_informado,0,não_informada,13342901.00,13342901.00,13342901.00
1,2012,5,64,secretaria_de_controle_e_desenvolvimento_urban...,64.01,empresa_de_urbanização_do_recife_-_urb/recife,4,despesas_de_capital,4,investimentos,...,64,6,s,3500561,construtora_queiroz_galvao_s/a,89,concorrencia,0.00,8649232.21,8649232.21
2,2013,12,61,secretaria_de_administração_e_gestão_de_pessoa...,61.03,fundo_financeiro_-_recifin,3,despesas_correntes,1,pessoal_e_encargos_sociais,...,280,0,n,1302469,fundo_financeiro_-_recifin,98,dispensado,6107163.18,6107163.18,6107163.18
3,2012,11,64,secretaria_de_controle_e_desenvolvimento_urban...,64.01,empresa_de_urbanização_do_recife_-_urb/recife,4,despesas_de_capital,4,investimentos,...,64,17,s,3500561,construtora_queiroz_galvao_s/a,89,concorrencia,0.00,0.00,5780250.85
4,2012,8,64,secretaria_de_controle_e_desenvolvimento_urban...,64.01,empresa_de_urbanização_do_recife_-_urb/recife,4,despesas_de_capital,4,investimentos,...,64,14,s,3500561,construtora_queiroz_galvao_s/a,89,concorrencia,0.00,4802110.04,4802110.04
5,2007,10,18,secretaria_de_saúde,18.01,secretaria_de_saúde_-_administração_direta,3,despesas_correntes,1,pessoal_e_encargos_sociais,...,796,90,x,0,credor_não_informado,0,não_informada,3767312.26,3767312.26,3767312.26
6,2017,6,48,secretaria_de_saúde_-_administração_supervisio...,48.01,fundo_municipal_de_saúde_-_fms,3,despesas_correntes,1,pessoal_e_encargos_sociais,...,2076,0,n,1302468,fundo_previdenciario_-_reciprev,98,dispensado,0.00,0.00,3362190.43
7,2017,6,48,secretaria_de_saúde_-_administração_supervisio...,48.01,fundo_municipal_de_saúde_-_fms,3,despesas_correntes,1,pessoal_e_encargos_sociais,...,2638,0,n,1302468,fundo_previdenciario_-_reciprev,98,dispensado,3319184.64,3319184.64,3319184.64
8,2007,8,18,secretaria_de_saúde,18.01,secretaria_de_saúde_-_administração_direta,3,despesas_correntes,1,pessoal_e_encargos_sociais,...,796,88,x,0,credor_não_informado,0,não_informada,3148991.07,3148991.07,3148991.07
9,2012,12,18,secretaria_de_saúde,18.01,secretaria_de_saúde_-_administração_direta,3,despesas_correntes,1,pessoal_e_encargos_sociais,...,656,92,x,0,credor_não_informado,0,não_informada,2997736.99,2997736.99,2997736.99
